In [ ]:
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Define paths
input_audio_path = "dataset/full_audio.wav"
output_dir = "dataset/audio_segments"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load the entire audiobook
audio = AudioSegment.from_wav(input_audio_path)

# Split on silence (adjust parameters as needed)
chunks = split_on_silence(audio, min_silence_len=1000, silence_thresh=-40)

# Ensure chunks are between 5 to 15 seconds
final_chunks = []
min_length = 5 * 1000  # 5 seconds in milliseconds
max_length = 15 * 1000  # 15 seconds in milliseconds

for chunk in chunks:
    if len(chunk) >= min_length:
        final_chunks.append(chunk)
    else:
        if final_chunks:
            final_chunks[-1] += chunk  # Append short chunk to the previous chunk
        else:
            final_chunks.append(chunk)  # Handle the case where the first chunk is too short

# Re-check to split longer chunks
final_chunks_split = []
for chunk in final_chunks:
    while len(chunk) > max_length:
        final_chunks_split.append(chunk[:max_length])
        chunk = chunk[max_length:]
    final_chunks_split.append(chunk)

# Export each chunk
for i, chunk in enumerate(final_chunks_split):
    chunk.export(os.path.join(output_dir, f"chunk_{i}.wav"), format="wav")

print(f"Exported {len(final_chunks_split)} chunks to {output_dir}")
